In [17]:
import cv2
import numpy as np
import onnxruntime as ort

In [18]:
def img_preprocessing(img):
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    img = cv2.resize(img, (256, 256))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img / 255.0
    for i in range(3):
        img[:, :, i] = (img[:, :, i] - mean[i]) / std[i]
    img = np.transpose(img, (2, 0, 1))
    return img

In [ ]:
def predict_on_camera(onnx_model_path,n_fps_pred=8,n_frames_pred=10,n_frames_split=2):
    #loader model
    
    session = ort.InferenceSession(onnx_model_path)
    input_name=session.get_inputs()[0].name
    output_name=session.get_outputs()[0].name

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return
    fps=cap.get(cv2.CAP_PROP_FPS)
    interval=int(fps/n_fps_pred)
    X=[]
    violent_frames=[]
    tmp_violent_frames=[]
    counts=0
    while True:
        counts+=1
        ret,frame=cap.read()
        if not ret:
            break
        if(counts%interval==0):
            counts=0
            processed_frame=img_preprocessing(frame)
            X.append(processed_frame)
            tmp_violent_frames.append(frame)
        input_data=np.array([X.copy()],dtype=np.float32)
        if(len(input_data[0])!=n_frames_pred):
            pass
        else:
            outputs = session.run([output_name], {input_name: input_data})
            y_preds=np.argmax(outputs[0],axis=1)[0]
            # print(outputs[0][0])
            if(y_preds==1):
                if(len(violent_frames)>=n_frames_pred):
                    violent_frames.extend(tmp_violent_frames[-(n_frames_pred-int(n_frames_pred/n_frames_split)+1):])
                else:
                    violent_frames.extend(tmp_violent_frames)
                print(y_preds)
            X=X[-int(n_frames_pred/n_frames_split)+1:]
            tmp_violent_frames=tmp_violent_frames[-int(n_frames_pred/n_frames_split)+1:]
        cv2.imshow('Video',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        if not ret:
                break
    cap.release()
    cv2.destroyAllWindows()

    return violent_frames
        

In [20]:
violentFrames=predict_on_camera(onnx_model_path=r'D:\DS\ViolenceDetection\notebooks\model_quantized.onnx',n_fps_pred=8,n_frames_pred=10)

[ 0.27917325 -0.48206955]
[ 1.1513852 -1.3709694]
[ 1.5113578 -1.7062737]
[ 1.4785805 -1.6747795]
[ 1.3493472 -1.5530142]
[ 0.86714095 -1.0771154 ]
[ 1.0036818 -1.221785 ]
[ 0.74582344 -0.96762097]
[ 0.31880853 -0.53091383]
[ 0.69210863 -0.91586554]
[ 0.7024844 -0.9221374]
[ 0.4756031 -0.6965579]
[ 0.70657545 -0.9341739 ]
[ 0.77434903 -0.9918822 ]
[ 0.33630836 -0.5514173 ]
[ 0.93896747 -1.1756968 ]
[ 1.8404858 -2.0027797]
[ 1.869554 -2.024868]
[ 1.2074715 -1.4186431]
[ 1.1257341 -1.3449188]
[ 1.3098286 -1.5146344]
[ 1.1956403 -1.4093943]
[ 0.30057934 -0.50598514]
[ 0.15725648 -0.34780037]
[ 0.5598162 -0.7809422]
[ 1.0064735 -1.2178031]


In [21]:
np.save('array.npy', violentFrames)

In [22]:
np.array(violentFrames).shape

(0,)

In [23]:
for i in range(len(violentFrames)):
    # Show each frame
    # if(i>=12):

    cv2.imshow('Video Frame', violentFrames[min(len(violentFrames)-1,i+11)])
    # else:
    #     cv2.imshow('Video Frame', violentFrames[i])
    
    # Wait for 25 ms between frames (40 FPS). Adjust as needed.
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break  # Press 'q' to exit early

cv2.destroyAllWindows()